# [Expectation Maximization](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm)

In statistics, an expectation–maximization (EM) algorithm is an iterative method to find maximum likelihood or maximum a posteriori (MAP) estimates of parameters in statistical models, where the model depends on unobserved latent variables. The EM iteration alternates between performing an expectation (E) step, which creates a function for the expectation of the log-likelihood evaluated using the current estimate for the parameters, and a maximization (M) step, which computes parameters maximizing the expected log-likelihood found on the E step. These parameter-estimates are then used to determine the distribution of the latent variables in the next E step.

* https://github.com/mcdickenson/em-gaussian
* https://github.com/sseemayer/mixem

In [ ]:
import numpy as np
import pandas as pd
import random as rand
import matplotlib.pyplot as plt
from scipy.stats import norm

In [ ]:
### Setup
# set random seed
rand.seed(42)

In [ ]:
# 2 clusters
# not that both covariance matrices are diagonal
mu1 = [0, 5]
sig1 = [ [2, 0], [0, 3] ]

mu2 = [5, 0]
sig2 = [ [4, 0], [0, 1] ]

In [ ]:
# generate samples
x1, y1 = np.random.multivariate_normal(mu1, sig1, 100).T
x2, y2 = np.random.multivariate_normal(mu2, sig2, 100).T

xs = np.concatenate((x1, x2))
ys = np.concatenate((y1, y2))
labels = ([1] * 100) + ([2] * 100)

data = {'x': xs, 'y': ys, 'label': labels}
df = pd.DataFrame(data=data)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
%matplotlib inline

In [ ]:
fig = plt.figure()
plt.scatter(data['x'], data['y'], 24, c=data['label'])
plt.show()
# fig.savefig("true-values.png")

In [ ]:
# initial guesses - intentionally bad
guess = { 'mu1': [1,1],
          'sig1': [ [1, 0], [0, 1] ],
          'mu2': [4,4],
          'sig2': [ [1, 0], [0, 1] ],
          'lambda': [0.4, 0.6]
        }

In [ ]:
# probability that a point came from a Guassian with given parameters
# note that the covariance must be diagonal for this to work
def prob(val, mu, sig, lam):
    p = lam
    for i in range(len(val)):
        p *= norm.pdf(val[i], mu[i], sig[i][i])
    return p

In [ ]:
# assign every data point to its most likely cluster
def expectation(dataFrame, parameters):
    for i in range(dataFrame.shape[0]):
        x = dataFrame['x'][i]
        y = dataFrame['y'][i]
    
        mu  = list(parameters['mu1'])
        sig = list(parameters['sig1'])
        lam = parameters['lambda'][0]
        p_cluster1 = prob([x, y], mu, sig, lam)
    
        mu  = list(parameters['mu2'])
        sig = list(parameters['sig2'])
        lam = parameters['lambda'][1]
        p_cluster2 = prob([x, y], mu, sig, lam)

        if p_cluster1 > p_cluster2:
            dataFrame['label'][i] = 1
        else:
            dataFrame['label'][i] = 2
    return dataFrame

In [ ]:
# update estimates of lambda, mu and sigma
def maximization(dataFrame, parameters):
    points1 = dataFrame[dataFrame['label'] == 1]
    points2 = dataFrame[dataFrame['label'] == 2]
    percent1 = len(points1) / float(len(dataFrame))
    percent2 = 1 - percent1
    parameters['lambda'] = [percent1, percent2 ]
    parameters['mu1'] = [points1['x'].mean(), points1['y'].mean()]
    parameters['mu2'] = [points2['x'].mean(), points2['y'].mean()]
    parameters['sig1'] = [ [points1['x'].std(), 0 ], [ 0, points1['y'].std() ] ]
    parameters['sig2'] = [ [points2['x'].std(), 0 ], [ 0, points2['y'].std() ] ]
    return parameters

In [ ]:
# get the distance between points
# used for determining if params have converged
def distance(old_params, new_params):
  dist = 0
  for param in ['mu1', 'mu2']:
    for i in range(len(old_params)):
      dist += (old_params[param][i] - new_params[param][i]) ** 2
  return dist ** 0.5

In [ ]:
# loop until parameters converge
shift = float('inf')
epsilon = 0.01
iters = 0
df_copy = df.copy()
# randomly assign points to their initial clusters
df_copy['label'] = map(lambda x: x+1, np.random.choice(2, len(df)))
params = pd.DataFrame(guess)

In [ ]:
while shift > epsilon:
    iters += 1
    
    # E-step
    updated_labels = expectation(df_copy.copy(), params)

    # M-step
    updated_parameters = maximization(updated_labels, params.copy())

    # see if our estimates of mu have changed
    # could incorporate all params, or overall log-likelihood
    shift = distance(params, updated_parameters)

    # logging
    print("iteration {}, shift {}".format(iters, shift))

    # update labels and params for the next iteration
    df_copy = updated_labels
    params = updated_parameters

    fig = plt.figure()
    plt.scatter(df_copy['x'], df_copy['y'], 24, c=df_copy['label'])
    # fig.savefig("iteration{}.png".format(iters))